# Take a picture and run the cloud model on it

## Prerequisites and cloud credential setup

In [9]:
import os
import glob

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

from picamera import PiCamera
from time import sleep
ENDPOINT = "https://westus2.api.cognitive.microsoft.com"

# Replace with a valid key
training_key = "aa9afbe3879c45d7aa2326c58867187d"
prediction_key = "1e72101a3c5644a58db5917251c6bb7d"
prediction_resource_id = "081e259d-c82a-4a49-acc6-27fc1adf8539"

from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

projectid="081e259d-c82a-4a49-acc6-27fc1adf8539"
# Now there is a trained endpoint that can be used to make a prediction
predictor = CustomVisionPredictionClient(prediction_key, endpoint=ENDPOINT)
trainer = CustomVisionTrainingClient(training_key,endpoint=ENDPOINT)


## Find a published iteration

In [10]:
iterations = trainer.get_iterations(projectid);
for iteration in iterations:
    print(iteration.name + "\t" + iteration.id + "\t" + iteration.status, end=' ' )
    if (iteration.publish_name):
        print("published under "+ iteration.publish_name, end=' ')
        publish_iteration_name = iteration.publish_name
    print(" ")
    
print("using "+ publish_iteration_name)


Iteration 1	b6fc2188-5f4a-4f64-b967-a5444707198c	Completed published under Iteration1  
using Iteration1


## Init camera

In [11]:

camera = PiCamera()

camera.resolution =(640,480)


print ("starting camera")
camera.start_preview()
sleep(5)


PiCameraMMALError: Failed to enable connection: Out of resources

## send the picture to the cloud for analysis and print the result

In [15]:
print ("capture")
camera.capture('/home/pi/camera/image.jpg')


print ("analyzing")
with open('/home/pi/camera/image.jpg',mode="rb") as test_data:
    results = predictor.classify_image(projectid, publish_iteration_name, test_data.read())
    for prediction in results.predictions:
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
        



capture
analyzing
	Tool: 100.00%
	Robot: 0.00%


## Exit

In [ ]:
camera.stop_preview()